# Cleaning Data in Python

## Common data problems

In this exercise, and throughout this chapter, you'll be working with bicycle ride sharing data in San Francisco called `ride_sharing`. It contains information on the start and end stations, the trip duration, and some user information for a bike sharing service.

The `user_type` column contains information on whether a user is taking a free ride and takes on the following values:
1. for free riders.
2. for pay per ride.
3. for monthly subscribers.
In this instance, you will print the information of `ride_sharing` using `.info()` and see a firsthand example of how an incorrect data type can flaw your analysis of the dataset.

In [1]:
# import pandas as pd
import pandas as pd

# import the ride sharing data
ride_sharing = pd.read_csv("https://raw.githubusercontent.com/xLiQuiDz/Erasmus/main/Data%20Science%20-%20Lecture%202/Datasets/ride_sharing.csv", index_col = 0)

# Print the information of ride_sharing
print(ride_sharing.info())

# Print summary statistics of user_type column
print(ride_sharing['user_type'].describe())

# Convert user_type from integer to category
ride_sharing['user_type_cat'] = ride_sharing['user_type'].astype('category')

# Write an assert statement confirming the change
assert ride_sharing['user_type_cat'].dtype == 'category'

# Print new summary statistics 
print(ride_sharing['user_type_cat'].describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25762 entries, 0 to 25759
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   duration         25762 non-null  object
 1   station_A_id     25762 non-null  int64 
 2   station_A_name   25762 non-null  object
 3   station_B_id     25762 non-null  int64 
 4   station_B_name   25762 non-null  object
 5   bike_id          25762 non-null  int64 
 6   user_type        25762 non-null  int64 
 7   user_birth_year  25762 non-null  int64 
 8   user_gender      25762 non-null  object
 9   user_type_cat    25762 non-null  int64 
 10  tire_sizes       25762 non-null  int64 
 11  duration_trim    25762 non-null  int64 
 12  duration_time    25762 non-null  int64 
 13  ride_id          25762 non-null  object
dtypes: int64(9), object(5)
memory usage: 2.9+ MB
None
count    25762.000000
mean         2.008307
std          0.704569
min          1.000000
25%          2.000000
50%   

In the previous exercise, you were able to identify that `category` is the correct data type for `user_type` and convert it in order to extract relevant statistical summaries that shed light on the distribution of `user_type`.

Another common data type problem is importing what should be numerical values as strings, as mathematical operations such as summing and multiplication lead to string concatenation, not numerical outputs.

In this exercise, you'll be converting the string column `duration` to the type `int`. Before that however, you will need to make sure to strip `"minutes"` from the column in order to make sure pandas reads it as numerical.

In [2]:
# Strip duration of minutes
ride_sharing['duration_trim'] = ride_sharing['duration'].str.strip("minutes").astype("int")

# Convert duration to integer
ride_sharing['duration_time'] = ride_sharing['duration_trim'].astype("int")

# Write an assert statement making sure of conversion
assert ride_sharing['duration_time'].dtype == 'int'

# Print formed columns and calculate average ride duration 
print(ride_sharing[['duration','duration_trim','duration_time']])
print(ride_sharing["duration_time"].mean())

         duration  duration_trim  duration_time
0      12 minutes             12             12
1      24 minutes             24             24
2       8 minutes              8              8
3       4 minutes              4              4
4      11 minutes             11             11
...           ...            ...            ...
25755  11 minutes             11             11
25756  10 minutes             10             10
25757  14 minutes             14             14
25758  14 minutes             14             14
25759  29 minutes             29             29

[25762 rows x 3 columns]
11.388906140827576


In this lesson, you're going to build on top of the work you've been doing with the `ride_sharing` DataFrame. You'll be working with the `tire_sizes` column which contains data on each bike's tire size.

Bicycle tire sizes could be either 26″, 27″ or 29″ and are here correctly stored as a categorical value. In an effort to cut maintenance costs, the ride sharing provider decided to set the maximum tire size to be 27″.

In this exercise, you will make sure the `tire_sizes` column has the correct range by first converting it to an integer, then setting and testing the new upper limit of 27″ for tire sizes.

In [3]:
# Convert tire_sizes to integer
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')

# Set all values above 27 to 27
ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27

# Reconvert tire_sizes back to categorical
ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')

# Print tire size description
print(ride_sharing['tire_sizes'].describe())

count     25762
unique        2
top          26
freq      13109
Name: tire_sizes, dtype: int64


A new update to the data pipeline feeding into `ride_sharing` has added the `ride_id` column, which represents a unique identifier for each ride. 

The update however coincided with radically shorter average ride duration times and irregular user birth dates set in the future. Most importantly, the number of rides taken has increased by 20% overnight, leading you to think there might be both complete and incomplete duplicates in the `ride_sharing` DataFrame.

In [4]:
# Find duplicates
duplicates = ride_sharing.duplicated("ride_id", keep = False)

# Sort your duplicated rides
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

# Print relevant columns of duplicated_rides
print(duplicated_rides[['ride_id','duration','user_birth_year']])

      ride_id    duration  user_birth_year
10329   10329   7 minutes             1970
10329   10329   7 minutes             1970
10566   10566  12 minutes             1992
10566   10566  12 minutes             1992
499       499   9 minutes             1986
499       499  34 minutes             1997
499       499  27 minutes             1991
499       499  59 minutes             1985


In the last exercise, you were able to verify that the new update feeding into `ride_sharing` contains a bug generating both complete and incomplete duplicated rows for some values of the `ride_id` column, with occasional discrepant values for the `user_birth_year` and `duration` columns.

In this exercise, you will be treating those duplicated rows by first dropping complete duplicates, and then merging the incomplete duplicate rows into one while keeping the average `duration`, and the minimum `user_birth_year` for each set of incomplete duplicate rows.

In [5]:
# Drop complete duplicates from ride_sharing
ride_dup = ride_sharing.drop_duplicates()

# Create statistics dictionary for aggregation function
statistics = {'user_birth_year': 'min', 'duration_trim': "mean"}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

## Text and categorical data problems

In this exercise and throughout this chapter, you'll be working with the `airlines` DataFrame which contains survey responses on the San Francisco Airport from airline customers.

The DataFrame contains flight metadata such as the airline, the destination, waiting times as well as answers to key questions regarding cleanliness, safety, and satisfaction. Another DataFrame named `categories` was created, containing all correct possible values for the survey columns. 

In [6]:
# Import the airlines data
airlines = pd.read_csv("https://raw.githubusercontent.com/xLiQuiDz/Erasmus/main/Data%20Science%20-%20Lecture%202/Datasets/airlines_final.csv", index_col = 0)

# Create a dictionary with all correct possible values
categories = {"cleanliness" : ["Clean", "Average", "Somewhat clean", "Somewhat dirty", "Dirty"],
             "safety": ["Neutral", "Very safe", "Somewhat safe", "Very unsafe", "Somewhat unsafe"],
             "satisfaction" : ["Very satisfied", "Neutral", "Somewhat satisfied", "Somewhat unsatisfied", "Very unsatisfied"]}

# Convert categories to a dataframe
categories = pd.DataFrame(categories)

# Print unique values of survey columns in airlines
print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
print('Safety: ', airlines['safety'].unique(), "\n")
print('Satisfaction: ', airlines['satisfaction'].unique(), "\n")

Cleanliness:  ['Clean' 'Average' 'Somewhat clean' 'Somewhat dirty' 'Dirty'] 

Safety:  ['Neutral' 'Very safe' 'Somewhat safe' 'Very unsafe' 'Somewhat unsafe'] 

Satisfaction:  ['Very satisfied' 'Neutral' 'Somewhat satsified' 'Somewhat unsatisfied'
 'Somewhat sasified' 'Very saisfied' 'Very unsatisfied'] 



Notice that in the `satisfaction` colomn there is one wrong category, "**Somewhat sasified**". We should tackle this inconsistency.

In [7]:
# Find the satisfaction category in airlines not in categories
cat_clean = set(airlines["satisfaction"]).difference(categories["satisfaction"])

# Find rows with that category
cat_clean_rows = airlines['satisfaction'].isin(cat_clean)

# Print rows with inconsistent category
print(airlines[cat_clean_rows])

# Print rows with consistent categories only
print(airlines[~cat_clean_rows])

        id        day         airline   destination    dest_region  dest_size  \
3     1157    Tuesday       SOUTHWEST   LOS ANGELES        West US        Hub   
4     2992  Wednesday        AMERICAN         MIAMI        East US        Hub   
6     2578   Saturday         JETBLUE    LONG BEACH        West US      Small   
9      919     Friday      AIR CANADA       TORONTO  Canada/Mexico        Hub   
11    1129    Tuesday       SOUTHWEST     SAN DIEGO        West US     Medium   
...    ...        ...             ...           ...            ...        ...   
2800  1942    Tuesday          UNITED        BOSTON        EAST US      Large   
2801  2130   Thursday  CATHAY PACIFIC     HONG KONG           Asia        Hub   
2803  2888  Wednesday          UNITED        AUSTIN     Midwest US     Medium   
2804  1475    Tuesday          ALASKA  NEW YORK-JFK        East US        Hub   
2808  2162   Saturday   CHINA EASTERN       QINGDAO           Asia      Large   

     boarding_area   dept_t

In this exercise, you will examine two categorical columns from this DataFrame, `dest_region` and `dest_size` respectively, assess how to address them and make sure that they are cleaned and ready for analysis.

In [8]:
# Print unique values of both columns
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

# Lower dest_region column and then replace "eur" with "europe"
airlines['dest_region'] = airlines['dest_region'].str.lower()
airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

# Remove white spaces from `dest_size`
airlines['dest_size'] = airlines['dest_size'].str.strip()

# Verify changes have been effected
print(airlines['dest_region'].unique())
print(airlines['dest_size'].unique())

['Asia' 'Canada/Mexico' 'West US' 'East US' 'Midwest US' 'EAST US'
 'Middle East' 'Europe' 'eur' 'Central/South America'
 'Australia/New Zealand' 'middle east']
['Hub' 'Small' '    Hub' 'Medium' 'Large' 'Hub     ' '    Small'
 'Medium     ' '    Medium' 'Small     ' '    Large' 'Large     ']
['asia' 'canada/mexico' 'west us' 'east us' 'midwest us' 'middle east'
 'europe' 'central/south america' 'australia/new zealand']
['Hub' 'Small' 'Medium' 'Large']


To better understand survey respondents from `airlines`, you want to find out if there is a relationship between certain responses and the day of the week and wait time at the gate.

The `airlines` DataFrame contains the `day` and `wait_min` columns, which are categorical and numerical respectively. The `day` column contains the exact day a flight took place, and `wait_min` contains the amount of minutes it took travelers to wait at the gate. To make your analysis easier, you want to create two new categorical variables:
- `wait_type`: `'short'` for 0-60 min, `'medium'` for 60-180 and `long` for 180+
- `day_week`: `'weekday'` if day is in the weekday, `'weekend'` if day is in the weekend.

In [9]:
# Import numpy as np
import numpy as np

# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                               labels = label_names)

# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)

## Advanced data problems
In this exercise and throughout this chapter, you will be working with a retail banking dataset stored in the `banking` DataFrame. The dataset contains data on the amount of money stored in accounts (`acct_amount`), their currency (`acct_cur`), amount invested (`inv_amount`), account opening date (`account_opened`), and last transaction date (`last_transaction`) that were consolidated from American and European branches.

You are tasked with understanding the average account size and how investments vary by the size of account, however in order to produce this analysis accurately, you first need to unify the currency amount into dollars.

In [10]:
banking = pd.read_csv("https://raw.githubusercontent.com/xLiQuiDz/Erasmus/main/Data%20Science%20-%20Lecture%202/Datasets/banking_dirty.csv", index_col = 0)

# Find values of acct_cur that are equal to 'euro'
acct_eu = banking['acct_cur'] == 'euro'

# Convert acct_amount where it is in euro to dollars
banking.loc[acct_eu, 'acct_amount'] = banking.loc[acct_eu, 'acct_amount'] * 1.1 

# Unify acct_cur column by changing 'euro' values to 'dollar'
banking.loc[acct_eu, 'acct_cur'] = 'dollar'

# Assert that only dollar currency remains
assert banking['acct_cur'].unique() == 'dollar'

After having unified the currencies of your different account amounts, you want to add a temporal dimension to your analysis and see how customers have been investing their money given the size of their account over each year. The `account_opened` column represents when customers opened their accounts and is a good proxy for segmenting customer activity and investment over time.

However, since this data was consolidated from multiple sources, you need to make sure that all dates are of the same format. You will do so by converting this column into a `datetime` object, while making sure that the format is inferred and potentially incorrect formats are set to missing. 

In [11]:
# Print the header of account_opened
print(banking["account_opened"].head())

# Convert account_opened to datetime
banking['account_opened'] = pd.to_datetime(banking['account_opened'],
                                           # Infer datetime format
                                           infer_datetime_format = True,
                                           # Return missing value for error
                                           errors = 'coerce') 

# Get year of account opened
banking['acct_year'] = banking['account_opened'].dt.strftime('%Y')

# Print acct_year
print(banking['acct_year'])

0    02-09-18
1    28-02-19
2    25-04-18
3    07-11-17
4    14-05-18
Name: account_opened, dtype: object
0     2018
1     2019
2     2018
3     2017
4     2018
      ... 
95    2018
96    2017
97    2017
98    2017
99    2017
Name: acct_year, Length: 100, dtype: object


New data has been merged into the `banking` DataFrame that contains details on how investments in the `inv_amount` column are allocated across four different funds A, B, C and D. 

Furthermore, the age and birthdays of customers are now stored in the `age` and `birth_date` columns respectively.

You want to understand how customers of different age groups invest. However, you want to first make sure the data you're analyzing is correct. You will do so by cross field checking values of `inv_amount` and `age` against the amount invested in different funds and customers' birthdays.

In [12]:
import datetime as dt

# Store fund columns to sum against
fund_columns = ['fund_A', 'fund_B', 'fund_C', 'fund_D']

# Find rows where fund_columns row sum == inv_amount
inv_equ = banking[["fund_A", "fund_B", "fund_C", "fund_D"]].sum(axis = 1) == banking["inv_amount"]

# Store consistent and inconsistent data
consistent_inv = banking[inv_equ]
inconsistent_inv = banking[~inv_equ]

# Store consistent and inconsistent data
print("Number of inconsistent investments: ", inconsistent_inv.shape[0])

Number of inconsistent investments:  10


Dealing with missing data is one of the most common tasks in data science. There are a variety of types of missingness, as well as a variety of types of solutions to missing data. You just received a new version of the `banking` DataFrame containing data on the amount held and invested for new and existing customers. However, there are rows with missing `inv_amount` values.

In [13]:
# Print number of missing values in banking
print(banking.isna().sum())

# Isolate missing and non missing values of inv_amount
missing_investors = banking[banking['inv_amount'].isna()]
investors = banking[~banking['inv_amount'].isna()]

cust_id             0
birth_date          0
Age                 0
acct_amount         0
inv_amount          2
fund_A              0
fund_B              0
fund_C              0
fund_D              0
account_opened      0
last_transaction    0
acct_cur            0
acct_year           0
dtype: int64
